In [1]:
import json
import pickle
import re
import collections
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np
from random import randint
import sys
import math
from scipy import spatial,sparse
import random

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
path_test2 = 'C:\\Users\\chenwang\\Documents\\courses\\FA16\\CS597\\twitterMining\\InitialSet\\Football\\location\\test2\\'
ff_path = path_test2 +'ff_of_rep\\'

In [3]:
with open(path_test2 + 'candidate_sp1.pickle','rb') as f:
    candidate_sp1 = pickle.load(f)
with open(path_test2 + 'candidate_sp2.pickle','rb+') as f:
    candidate_sp2 = pickle.load(f)
with open(path_test2 + 'represent_id.pickle','rb') as f:
    feature = pickle.load(f)
with open(path_test2 + 'location_list_sp1.pickle','rb') as f:
    location_sp1 = pickle.load(f)
with open(path_test2 + 'location_list_sp2.pickle','rb') as f:
    location_sp2 = pickle.load(f)

# construct a vector

In [4]:
def construct_vector(candidate_list,feature):
    
    V = dict((c,np.zeros(len(feature),)) for c in candidate_list)

    for i in range(len(feature)):  
        with open(ff_path + str(feature[i]) + '.pickle','rb') as f:

                candidate = pickle.load(f)

                for c in candidate:
                    if c in V.keys():
                        #print('true')
                        V[c][i] = 1 
    return V

In [5]:
Vector1 = construct_vector(candidate_sp1,feature)
Vector2 = construct_vector(candidate_sp2,feature)

# normal labeling process

In [6]:
chicago = ['chicago','Arlington Heights', 'Aurora', 'Berwyn', 'Bolingbrook', 
            'Cicero','Des Plaines', 
           'Elgin','Evanston', 'Gary', 'Hammond',
            'Hoffman Estates', 'Joliet', 'Kenosha', 'Mount Prospect','Naperville', 
            'Oak Lawn', 'Oak Park', 'Orland Park',
            'Palatine', 'Schaumburg','Skokie','Tinley Park','Waukegan','Wheaton']

In [15]:
def labeling(location_sp,seed=0,sample_mode='normal',area = chicago):
    
    # select those provide labels
    candidate_w_loc = []
    for loc in location_sp.keys():
        if location_sp[loc] != 'NaN':
            candidate_w_loc.append((loc,location_sp[loc].lower())) 
    
    if sample_mode != 'None':
        random.seed(seed)
        if sample_mode == 'nomral' or sample_mode == 'undersample':
            train = random.sample(set(candidate_w_loc),1000) # random sample (try oversampling, undersampling)
        elif sample_mode == 'oversample':
            train = random.sample(set(candidate_w_loc),3000)
        
        with open(path_test2 + sample_mode + '_train.pickle','wb+') as f: # save
            pickle.dump(train,f)

        df_train = pd.DataFrame(train,columns = ['id','location'])
    
    else:
        df_train = pd.DataFrame(candidate_w_loc,columns = ['id','location'])
    
    df_train['label']=0
    for chi in area:
        df_train.loc[df_train.location.str.contains(chi.lower()), 'label'] = 1 
        
    # save the id of these temp 1 labeled to a list, query twitter user_timeline to see their tweets
    training_chicago_id = df_train[df_train.label ==1]['id'].tolist()
   
    with open(path_test2 + sample_mode + '_training_id.pickle','wb') as f:
        pickle.dump(training_chicago_id,f)
        
    return df_train

In [84]:
df_train = labeling(location_sp1,seed=0, sample_mode='oversample',area = chicago)
print('pos',len(df_train[df_train.label==1]))
print('neg',len(df_train[df_train.label==0]))

with open(path_test2 + 'oversample_train_no_match.pickle','rb') as f: #open no information
    no_match = pickle.load(f) 
with open(path_test2 + 'oversample_train_no_info.pickle','rb') as f: #open no information
    no_info = pickle.load(f) 
    
df_train.loc[df_train['id'].isin(no_match),'label'] = 0
df_train = df_train.loc[~(df_train['id'].isin(no_info))] # remove those with no information available 
postive_num = len(df_train[df_train['label']==1])

print('pos',len(df_train[df_train.label==1]))
print('neg',len(df_train[df_train.label==0]))

pos 853
neg 2147
pos 579
neg 2334


# hand labeling testing data

In [22]:
label_test = labeling(location_sp2,seed=0, sample_mode='None', area = chicago)
print('pos',len(label_test[label_test.label==1]))
print('neg',len(label_test[label_test.label==0]))

pos 1475
neg 3897


In [23]:
with open(path_test2 + 'None_train_no_match.pickle','rb') as f: #open no information
    no_match = pickle.load(f) 
with open(path_test2 + 'None_train_no_info.pickle','rb') as f: #open no information
    no_info = pickle.load(f) 
    
print('pos',len(label_test[label_test.label==1]))
print('neg',len(label_test[label_test.label==0]))

label_test.loc[label_test['id'].isin(no_match),'label'] = 0
label_test = label_test.loc[~(label_test['id'].isin(no_info))] # remove those with no information available 
postive_num = len(label_test[label_test['label']==1])    

print('pos',len(label_test[label_test.label==1]))
print('neg',len(label_test[label_test.label==0]))

pos 1475
neg 3897
pos 827
neg 4158


In [24]:
label_test_id = label_test['id'].tolist()
label_test_id[:10]

[150852950,
 1508556810,
 177307659,
 744386991542964224,
 57294861,
 4384669709,
 724009354303836160,
 3305127954,
 388612119,
 32981017]

In [69]:
# tricks: remove some negative samples to create a more balanced set (undersampling?)
reject_id = random.sample(set(df_train[df_train.label ==0]['id'].tolist()),2334-3*579)
df_train = df_train.loc[~(df_train['id'].isin(reject_id))] 

In [83]:
df_train[df_train.label==0].shape

(2334, 3)

# setting up training sets

In [85]:
y_train = df_train['label'].tolist()
id_train = df_train['id'].tolist()

x_train = []
for id in id_train:
    x_train.append(Vector1[id])

# setting up testing sets

In [86]:
x_test = []
for id in candidate_sp2:
    x_test.append(Vector2[id])

# classification

In [87]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
import sklearn.metrics as mt

In [88]:
def classification(clf, X_train, Y_train, X_test,id_test):
    clf.fit(X_train,Y_train)
    target_test = clf.predict(X_test)
    df_y_test = pd.DataFrame(id_test,columns=['id'])
    df_y_test['label'] = target_test
    rel_id_test = df_y_test[df_y_test['label']==1]['id'].tolist()
    irrel_id_test = df_y_test[df_y_test['label']==0]['id'].tolist()
    
    return df_y_test,rel_id_test,irrel_id_test

In [89]:
def scores(df, label_test):
    df_compare = df[df['id'].isin(label_test_id)]
    df_combined = pd.merge(df_compare, label_test, on='id', how='outer')
    
    precision = mt.precision_score(df_combined['label_y'],df_combined['label_x'])
    recall = mt.recall_score(df_combined['label_y'],df_combined['label_x'])
    f1 = mt.f1_score(df_combined['label_y'],df_combined['label_x'])
    accuracy = mt.accuracy_score(df_combined['label_y'],df_combined['label_x'])
    
    print('precision = ', precision)
    print('recall = ', recall)
    print('F1 = ', f1)
    print('accuracy = ', accuracy)
    
    return [precision, recall, f1, accuracy]
    

In [90]:
new_path = 'C:/Users/chenwang/Documents/courses/FA16/CS597/twitterMining/InitialSet/Football/location/test2/undersampling/'

In [91]:
clf = BernoulliNB()
df,rel_id,irrel_id = classification(clf, x_train,y_train,x_test,candidate_sp2)
with open(new_path + 'clf_NB_test_rel.pickle','wb+') as f:
    pickle.dump(rel_id,f)
with open(new_path + 'clf_NB_test_irrel.pickle','wb+') as f:
    pickle.dump(irrel_id,f)

scores(df,label_test)

precision =  0.274440518257
recall =  0.281741233374
F1 =  0.278042959427
accuracy =  0.757271815446


[0.27444051825677268,
 0.28174123337363965,
 0.27804295942720758,
 0.75727181544633904]

In [92]:
clf = DecisionTreeClassifier()
df,rel_id,irrel_id = classification(clf,x_train,y_train,x_test,candidate_sp2)
with open(new_path + 'clf_DT_test_rel.pickle','wb+') as f:
    pickle.dump(rel_id,f)
with open(new_path + 'clf_DT_test_irrel.pickle','wb+') as f:
    pickle.dump(irrel_id,f)

scores(df,label_test)

precision =  0.301310043668
recall =  0.250302297461
F1 =  0.273447820343
accuracy =  0.779338014042


[0.30131004366812225,
 0.25030229746070132,
 0.27344782034346099,
 0.77933801404212633]

In [93]:
clf = KNeighborsClassifier(n_neighbors=1)
df,rel_id,irrel_id = classification(clf,x_train,y_train,x_test,candidate_sp2)
with open(new_path + 'clf_KNN_test_rel.pickle','wb+') as f:
    pickle.dump(rel_id,f)
with open(new_path + 'clf_KNN_test_irrel.pickle','wb+') as f:
    pickle.dump(irrel_id,f)
    
scores(df,label_test)

precision =  0.276737967914
recall =  0.250302297461
F1 =  0.262857142857
accuracy =  0.767101303912


[0.2767379679144385,
 0.25030229746070132,
 0.26285714285714284,
 0.76710130391173525]

In [94]:
clf = GradientBoostingClassifier(n_estimators=100,learning_rate=1, max_depth=1, random_state=10)
df,rel_id,irrel_id = classification(clf,x_train,y_train,x_test,candidate_sp2)
with open(new_path + 'clf_GBC_test_rel.pickle','wb+') as f:
    pickle.dump(rel_id,f)
with open(new_path + 'clf_GBC_test_irrel.pickle','wb+') as f:
    pickle.dump(irrel_id,f)
    
scores(df,label_test)

precision =  0.359649122807
recall =  0.09915356711
F1 =  0.155450236967
accuracy =  0.821263791374


[0.35964912280701755,
 0.09915356711003627,
 0.15545023696682464,
 0.82126379137412242]

In [95]:
# clf = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(15,), random_state=1)
clf = MLPClassifier(hidden_layer_sizes=(100,), learning_rate = 'adaptive', random_state = 0,max_iter=1000)
df,rel_id,irrel_id = classification(clf,x_train,y_train,x_test,candidate_sp2)
with open(new_path + 'clf_NNW_test_rel.pickle','wb+') as f:
    pickle.dump(rel_id,f)
with open(new_path + 'clf_NNW_test_irrel.pickle','wb+') as f:
    pickle.dump(irrel_id,f)
    
scores(df,label_test)

precision =  0.326923076923
recall =  0.205562273277
F1 =  0.252412769117
accuracy =  0.797993981946


[0.32692307692307693,
 0.20556227327690446,
 0.2524127691165553,
 0.79799398194583748]

# questions: 

begining of Oct

1. for initial list:
I applied very stringent approach (only look at user profile who provides 'chicago' as their location). Reasonable? Extra learning required?

2. feature selection:
How many representative nodes do we need?

3. feature ranking:
My approach reasonable? Any improvement?

4. rate limit problem
when use the clf on test data, need to retrieve test nodes's friends_list. thus only can test a small portion once a time.
can't be real time very slow

5. is this approach generic enough? 
chicago is famous. What if it's non-famous? what if it's smaller or bigger area?

6. How to utilize the query user by keyword?

7. How to test the performance? 
We hae ground truth (a small portion of user will list their profile location <10%)

# Things to do:
a. SVD rank feature 

b. haven\'t add prior to naive bayes

c. think!
Difference of Amin's goal and my goal
he wants to find interest, i want to find location. I have more information than he has. How can I utilize these information.

# estimate prior
<img src= 'prior.png'>

i feel like this prior wont affect ranking so can be omit, doesnt change with Ux